In [1]:
from collections import defaultdict
import pandas as pd
from surprise import Reader, Dataset
from surprise import KNNWithMeans, KNNBasic, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate

import matplotlib.pyplot as plt
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
class RefinedMyAlgo():
    def __init__(self, rating_data='', data_frame='', movie_data=''):
        if rating_data:
            reader = Reader(line_format='user item rating timestamp', sep=',')
            self.ratings = Dataset.load_from_file(rating_data, reader)
#             self.trainset, self.testset = train_test_split(self.ratings, test_size=0.25)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
        elif not data_frame.empty:
            reader = Reader(rating_scale=(0, 5))
            self.ratings = Dataset.load_from_df(data_frame[['userId', 'movieId', 'rating']], reader)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            
        if movie_data:
            self.movies = pd.read_csv(movie_data, low_memory=False)
            self.movies['year'] = self.movies['title'].apply(lambda x: x[-5:-1])
            self.movies['title'] = self.movies['title'].apply(lambda x: x[:-7])
            self.movies['genres'] = self.movies['genres'].apply(lambda x: x.replace('|',', '))

        
    def set_k(self, k_value=''):
        if k_value:
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            self.algo = algo
            self.algo.fit(self.trainset)
        else:
            algo = SVD()
            self.algo = algo
            self.algo.fit(self.trainset)
        
        
    def find_best_k(self, k_value=''):
        if k_value:
            print('K = {}'.format(k_value))
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            return cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=True)
        else:
            aux = []
            for k_value in [3, 5, 7, 10, 15, 20, 30, 40]:
                print('K = {}'.format(k_value))
                algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
                my_dict = cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=False)
                my_dict['k_value'] = k_value
                aux.append(my_dict)
            return aux
    
    
    def set_testset(self, users):
        if users:
            user_ratings = self.trainset.ur
            movies_ids = list(self.movies['movieId'])
            global_mean=self.trainset.global_mean
            my_testset = []
            
            for user in users:
                iuid = self.trainset.to_inner_uid(str(user))
                for movie in movies_ids:
                    is_in = False
                    for rating in user_ratings[iuid]:
#                         print( 'MOVIE: {}, RATING: {}'.format(movie,bla.trainset.to_raw_iid(rating[0])) )
                        if int(movie) == int(self.trainset.to_raw_iid(int(rating[0]))):
                            is_in = True
                            break
                    if not is_in:
                        my_tuple = (str(user),str(movie),global_mean)
                        my_testset.append(my_tuple)
                        
            self.testset = my_testset
        else:
            testset = self.trainset.build_anti_testset()
            self.testset = testset
        return self.testset


    def predict_ratings(self,users=''):
        # # Predict ratings for all pairs (u, i) that are NOT in the training set.
#         testset = self.trainset.build_anti_testset()
#         self.testset = testset
        testset = self.set_testset(users)
        predictions = self.algo.test(testset)
        self.predictions = predictions
        
        
    def set_perfil_movies(self, users):
        metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
        metadata = metadata.drop(columns="timestamp")

        metadata_filtered = metadata[metadata.userId.isin(users)]

        self.group_sparse_mtx = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
        
        self.perfil_movies = list(self.group_sparse_mtx)
        
    
    ### You must call self.set_perfil_movies() before
    def set_candidate_movies(self):
        candidate_movies = []
        for item in refinedMyAlgo.movies.iterrows():
        #     get the movieId of each movie in movies dataframe
            if item[1].values[0] not in self.perfil_movies:
                candidate_movies.append(item[1].values[0])
        self.candidate_movies = candidate_movies
        
        
    def calc_similarity_matrix(self):
        #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
        tfidf = TfidfVectorizer(stop_words='english')
        
        #Replace NaN with an empty string
        self.movies['title'] = self.movies['title'].fillna('')
        self.movies['genres'] = self.movies['genres'].fillna('')
        
        #Construct the required TF-IDF matrix by fitting and transforming the data
        tfidf_matrix_title = tfidf.fit_transform(self.movies['title'])
        tfidf_matrix_genres = tfidf.fit_transform(self.movies['genres'])
        
        #Compute the cosine similarity matrix
        self.cosine_sim_movies_title = cosine_similarity(tfidf_matrix_title, tfidf_matrix_title)
        self.cosine_sim_movies_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)
        
        
    def get_similar_movies(self, references, title_weight=0.5):
        recs = []
        for movie in references:
            # Get the pairwsie similarity scores of all movies with that movie
            movie_idx = int(self.movies[self.movies['movieId']==movie['movieID']].index[0])
            sim_scores_title = list(enumerate(self.cosine_sim_movies_title[movie_idx]))
            sim_scores_genres = list(enumerate(self.cosine_sim_movies_genres[movie_idx]))
            
            # Calculate total similarity based on title and genres
            total_sim_score = []
            for i in range(len(sim_scores_title)):
                print("sim_score_title= {}\t sim_score_genres= {}".format(sim_scores_title[i][1], sim_scores_genres[i][1]))
                aux = (sim_scores_title[i][1]*title_weight) + (sim_scores_genres[i][1]*(1-title_weight))
                total_sim_score.append((i, aux))
                print("sim_score_total= {}".format(total_sim_score))
                
            # Sort the movies based on the similarity scores
            total_sim_score = sorted(total_sim_score, key=lambda x: x[1], reverse=True)
            self.total_sim_score = total_sim_score
            
            candidates_sim_score = []
            for item in total_sim_score:
                if self.movies.loc[item[0]].values[0] not in self.perfil_movies:
                    candidates_sim_score.append(item)
            
            # Get the scores of the 10 most similar movies
            candidates_sim_score = candidates_sim_score[1:11]
            
            recs.append(candidates_sim_score)
            
        return recs
    
    
    def get_relevance_score(self, recs, references):
        count = 0
        recs_dict = []
        for reference in references:
        #     print('Referência: {}\t gêneros: {}'.format(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][1], refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][2]))

            for movie in recs[count]:
                aux = {}

                movie_id = self.movies.loc[movie[0]].values[0]
                movie_title = self.movies.loc[movie[0]].values[1]
                movie_genres = self.movies.loc[movie[0]].values[2]
                movie_similarity = movie[1]
                movie_relevance = ((reference['rating']/5.0)+movie_similarity)/2

                aux['movie_id'] = movie_id
                aux['movie_title'] = movie_title
                aux['movie_genres'] = movie_genres
                aux['movie_similarity'] = movie_similarity
                aux['movie_relevance'] = movie_relevance

                recs_dict.append(aux)

        #         print('\tSim: {},\trelevance: {},\tmovieId: {},\ttitle: {}'.format(aux['movie_similarity'], aux['movie_relevance'], aux['movie_id'], aux['movie_title']))

            count=count+1

        recs_dict = sorted(recs_dict, key = lambda i: i['movie_relevance'],reverse=True)

        return recs_dict
    
    
    def calc_distance_item_in_list(self, item, this_list, title_weight=0.5):

        idx_i = int(self.movies[self.movies['movieId']==item['movie_id']].index[0])

        total_dist = 0
        for movie in this_list:
            
            idx_j = int(self.movies[self.movies['movieId']==int(movie['movie_id'])].index[0])

            sim_i_j = (self.cosine_sim_movies_title[idx_i][idx_j]*title_weight) + (self.cosine_sim_movies_genres[idx_i][idx_j]*(1-title_weight))
            dist_i_j = 1 - sim_i_j
            total_dist = total_dist + dist_i_j

        result = total_dist/len(this_list)
        return result
    
    
    def calc_diversity_score(self, actual_list, candidates_list, alfa=0.5):
        '''
        This function implemented here was based on MARIUS KAMINSKAS and DEREK BRIDGE paper: Diversity, Serendipity, Novelty, and Coverage: A Survey and Empirical Analysis of Beyond-Accuracy Objectives in Recommender Systems
        func(i,R) = (relevance[i]*alfa) + (dist_i_R(i,R)*(1-alfa))
        '''
        diversity_score = []
        count = 0

        for item in candidates_list:

            aux = {}
            dist_item_R = self.calc_distance_item_in_list(item=item, this_list=actual_list)
            aux['div_score'] = (item['movie_relevance']*alfa) + (dist_item_R*(1-alfa))
            aux['idx'] = count
            diversity_score.append(aux)
            count = count + 1

        return diversity_score
    
    
    def diversify_recs_list(self, recs, n=10):
        '''
        This function implemented here was based on MARIUS KAMINSKAS and DEREK BRIDGE paper: Diversity, Serendipity, Novelty, and Coverage: A Survey and Empirical Analysis of Beyond-Accuracy Objectives in Recommender Systems
        The Greedy Reranking Algorithm.
        '''
        diversified_list = []
        
        while len(diversified_list) < n:
            if len(diversified_list) == 0:
                diversified_list.append(recs[0])
                recs.pop(0)
            else:
                diversity_score = self.calc_diversity_score(actual_list=diversified_list, candidates_list=recs)
                diversity_score = sorted(diversity_score, key = lambda i: i['div_score'],reverse=True)
#               #  Add the item that maximize diversity in the list 
                item = diversity_score[0]
                diversified_list.append(recs[item['idx']])
#               #  Remove this item from the candidates list
                recs.pop(item['idx'])
    
        return diversified_list
                

In [3]:
refinedMyAlgo = RefinedMyAlgo(rating_data='ml-latest-small/ratings.csv', movie_data='ml-latest-small/movies.csv')
refinedMyAlgo.set_k()

In [4]:
my_users = [77,596,452,243,420]

refinedMyAlgo.predict_ratings(users=my_users)
len(refinedMyAlgo.predictions)

47891

In [5]:
# metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
# metadata = metadata.drop(columns="timestamp")

# metadata_filtered = metadata[metadata.userId.isin(my_users)]

# my_group_sparse = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
# my_group_sparse.head()

refinedMyAlgo.set_perfil_movies(users=my_users)
refinedMyAlgo.set_candidate_movies()

# print(refinedMyAlgo.perfil_movies)
# print(refinedMyAlgo.candidate_movies)
refinedMyAlgo.group_sparse_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,0,0,0.0,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
243,0,5,0.0,0,4,0,4,0,4,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
420,4,0,3.5,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
452,0,4,0.0,0,0,0,4,5,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
596,4,0,3.5,4,0,4,0,0,0,3.5,...,2.5,3.5,3.5,4,3.5,3.5,3.5,3.5,4,4


In [6]:
group_filled_mtx = refinedMyAlgo.group_sparse_mtx.copy()

for index, row in group_filled_mtx.iterrows():
    for col in list(group_filled_mtx):
        if(group_filled_mtx.loc[index,col] == 0.0):
            aux = list(filter(lambda x: x.uid==str(index) and x.iid==str(col), refinedMyAlgo.predictions))
            group_filled_mtx.loc[index,col] = aux[0].est

group_filled_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,4.040313,3.481926,3.953764,4.205609,3.637876,3.317135,2.431438,4.059315,3.132455,4.401582,...,2.995072,3.228084,3.518264,3.594154,3.349753,3.517887,3.438034,3.295381,3.387541,3.817185
243,4.295327,5.000000,4.376261,4.240133,4.000000,4.328648,4.000000,4.565318,4.000000,4.474861,...,3.961716,4.144488,4.215426,4.381747,3.943771,4.243491,4.280943,4.020823,4.193527,4.461219
420,4.000000,3.637428,3.500000,3.810122,3.785526,3.511038,2.571069,4.073663,3.290671,4.623903,...,3.300680,3.275672,3.576611,3.594179,3.471851,3.668482,3.626263,3.658623,3.646182,3.640692
452,4.623341,4.000000,4.605985,4.579803,4.908377,4.656290,4.000000,5.000000,4.056835,5.000000,...,4.451087,4.328133,4.531907,4.498232,4.386802,4.409529,4.616640,4.355252,4.670289,4.480946
596,4.000000,3.309253,3.500000,4.000000,3.660753,4.000000,2.759431,3.966246,3.194493,3.500000,...,2.500000,3.500000,3.500000,4.000000,3.500000,3.500000,3.500000,3.500000,4.000000,4.000000


In [7]:
########################################################################
# # Implementing least misery ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    values.append( float(min(my_col)) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

agg_group_perf.head()

,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
900,4,3.30925,3.5,3.81012,3.63788,3.31714,2.43144,3.96625,3.13245,3.5,...,2.5,3.22808,3.5,3.59415,3.34975,3.5,3.43803,3.29538,3.38754,3.64069


In [8]:
group_pref_dict = []
for col in list(agg_group_perf):
    my_dict = {}
#     print('Valor: {}, Coluna: {}'.format(agg_group_perf.loc[900,col], col))
    my_dict['rating'] = agg_group_perf.loc[900,col]
    my_dict['movieID'] = col
    group_pref_dict.append(my_dict)
    
group_pref_dict = sorted(group_pref_dict, key = lambda i: i['rating'],reverse=True)
group_pref_dict

[{'rating': 4.18124850177708, 'movieID': 38061},
 {'rating': 4.129456713245073, 'movieID': 5618},
 {'rating': 4.104624554399897, 'movieID': 57669},
 {'rating': 4.087483370919955, 'movieID': 51255},
 {'rating': 4.06409317012724, 'movieID': 6807},
 {'rating': 4.063324392246618, 'movieID': 60069},
 {'rating': 4.060108722849658, 'movieID': 168252},
 {'rating': 4.047061605021381, 'movieID': 122882},
 {'rating': 4.038739830869619, 'movieID': 76093},
 {'rating': 4.0319890406007, 'movieID': 1394},
 {'rating': 4.027765605054068, 'movieID': 6350},
 {'rating': 4.01771405294984, 'movieID': 904},
 {'rating': 4.017220094901727, 'movieID': 3703},
 {'rating': 4.006379981664054, 'movieID': 31658},
 {'rating': 4.00169141634156, 'movieID': 48394},
 {'rating': 4.0, 'movieID': 1},
 {'rating': 4.0, 'movieID': 260},
 {'rating': 4.0, 'movieID': 296},
 {'rating': 4.0, 'movieID': 593},
 {'rating': 4.0, 'movieID': 750},
 {'rating': 4.0, 'movieID': 1028},
 {'rating': 4.0, 'movieID': 1035},
 {'rating': 4.0, 'movie

In [9]:
refinedMyAlgo.calc_similarity_matrix()

In [10]:
references = group_pref_dict[0:10]
# references = group_pref_dict

for item in references:
    print(item)
# print(references)

{'rating': 4.18124850177708, 'movieID': 38061}
{'rating': 4.129456713245073, 'movieID': 5618}
{'rating': 4.104624554399897, 'movieID': 57669}
{'rating': 4.087483370919955, 'movieID': 51255}
{'rating': 4.06409317012724, 'movieID': 6807}
{'rating': 4.063324392246618, 'movieID': 60069}
{'rating': 4.060108722849658, 'movieID': 168252}
{'rating': 4.047061605021381, 'movieID': 122882}
{'rating': 4.038739830869619, 'movieID': 76093}
{'rating': 4.0319890406007, 'movieID': 1394}


In [ ]:
# references=[904, 318, 38061, 31658, 57669]
recs = refinedMyAlgo.get_similar_movies(references)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




sim_score_total= [(0, 0.4103439199877791), (1, 0.3048104255254095), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.16169690011806773), (8, 0.0), (9, 0.1598740623563269), (10, 0.0), (11, 0.0), (12, 0.32170748741296346), (13, 0.0), (14, 0.15677736108887924), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.2310897489765119), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.33710681949381877), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.15557702424682054), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.2184556570518338), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.3048104255254095), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.0), (67, 0.0), (68, 0.0), (69, 0.0), (70, 0.0), (71, 0.0), (72, 0.0), (73, 0.0), (74, 0.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



sim_score_total= [(0, 0.13379323844570007), (1, 0.0), (2, 0.28545770321995495), (3, 0.25250771985025183), (4, 0.5), (5, 0.0), (6, 0.28545770321995495), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.25250771985025183), (11, 0.254774954753328), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.5), (18, 0.5), (19, 0.17611689597495786), (20, 0.2164918282091859), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.2341593743242762), (35, 0.28545770321995495), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.2607617603730761), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.25250771985025183), (48, 0.0), (49, 0.2341593743242762), (50, 0.0), (51, 0.0), (52, 0.25250771985025183), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.2668674764506304), (57, 0.28545770321995495), (58, 0.5), (59, 0.0), (60, 0.28545770321995495), (61, 0.5), (62, 0.18198970341941656), (63, 0.0), (64, 0.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
print(refinedMyAlgo.total_sim_score)

[(9463, 1.0), (1904, 0.8174867105895136), (507, 0.5), (595, 0.5), (1056, 0.5), (1261, 0.5), (1767, 0.5), (1906, 0.5), (1907, 0.5), (1908, 0.5), (1909, 0.5), (1921, 0.5), (1987, 0.5), (2113, 0.5), (2114, 0.5), (2683, 0.5), (2760, 0.5), (2843, 0.5), (3369, 0.5), (3598, 0.5), (3694, 0.5), (4459, 0.5), (4469, 0.5), (4685, 0.5), (5080, 0.5), (5273, 0.5), (5582, 0.5), (5604, 0.5), (5997, 0.5), (6770, 0.5), (6932, 0.5), (7268, 0.5), (7985, 0.5), (8424, 0.5), (8456, 0.5), (9150, 0.5), (9403, 0.5), (9687, 0.5), (9722, 0.5), (439, 0.4740052347801208), (1905, 0.4740052347801208), (2001, 0.4740052347801208), (3926, 0.4740052347801208), (8322, 0.4740052347801208), (9065, 0.4740052347801208), (9474, 0.4740052347801208), (285, 0.469920106724225), (1071, 0.469920106724225), (1183, 0.469920106724225), (2038, 0.469920106724225), (3885, 0.469920106724225), (5534, 0.469920106724225), (6726, 0.469920106724225), (7616, 0.469920106724225), (7855, 0.469920106724225), (8202, 0.469920106724225), (8314, 0.469920

In [16]:
amarelo = references
vermelho = recs

count = 0
recs_dict = []
for reference in amarelo:
#     print('Referência: {}\t gêneros: {}'.format(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][1], refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][2]))

    for movie in vermelho[count]:
        aux = {}

        movie_id = refinedMyAlgo.movies.loc[movie[0]].values[0]
        movie_title = refinedMyAlgo.movies.loc[movie[0]].values[1]
        movie_genres = refinedMyAlgo.movies.loc[movie[0]].values[2]
        movie_similarity = movie[1]
        movie_relevance = ((reference['rating']/5.0)+movie_similarity)/2

        aux['movie_id'] = movie_id
        aux['movie_title'] = movie_title
        aux['movie_genres'] = movie_genres
        aux['movie_similarity'] = movie_similarity
        aux['movie_relevance'] = movie_relevance

        recs_dict.append(aux)

        print('\tSim: {},\trelevance: {},\tmovieId: {},\ttitle: {}'.format(aux['movie_similarity'], aux['movie_relevance'], aux['movie_id'], aux['movie_title']))

    count=count+1

recs_dict = sorted(recs_dict, key = lambda i: i['movie_relevance'],reverse=True)

print("@ @ @ @ @ @ @ @ @ @ @ @ @")
recs_dict

	Sim: 0.5000000000000001,	relevance: 0.6827000304811133,	movieId: 2413,	title: Clue
	Sim: 0.5000000000000001,	relevance: 0.6827000304811133,	movieId: 4610,	title: January Man, The
	Sim: 0.5000000000000001,	relevance: 0.6827000304811133,	movieId: 4898,	title: Novocaine
	Sim: 0.5000000000000001,	relevance: 0.6827000304811133,	movieId: 5021,	title: Murder by Death
	Sim: 0.5000000000000001,	relevance: 0.6827000304811133,	movieId: 5938,	title: Deathtrap
	Sim: 0.47828245907659095,	relevance: 0.6718412600194088,	movieId: 1799,	title: Suicide Kings
	Sim: 0.47828245907659095,	relevance: 0.6718412600194088,	movieId: 27674,	title: 11:14
	Sim: 0.471993028450314,	relevance: 0.6686965447062703,	movieId: 1086,	title: Dial M for Murder
	Sim: 0.471993028450314,	relevance: 0.6686965447062703,	movieId: 1661,	title: Switchback
	Sim: 0.471993028450314,	relevance: 0.6686965447062703,	movieId: 2579,	title: Following
	Sim: 0.5,	relevance: 0.6808348486346194,	movieId: 257,	title: Just Cause
	Sim: 0.5,	relevanc

[{'movie_id': 3280,
  'movie_title': 'Baby, The',
  'movie_genres': 'Horror',
  'movie_similarity': 0.6572141853842605,
  'movie_relevance': 0.7433642685875197},
 {'movie_id': 93838,
  'movie_title': 'The Raid: Redemption',
  'movie_genres': 'Action, Crime',
  'movie_similarity': 0.5639156906809616,
  'movie_relevance': 0.6944099556775261},
 {'movie_id': 2413,
  'movie_title': 'Clue',
  'movie_genres': 'Comedy, Crime, Mystery, Thriller',
  'movie_similarity': 0.5000000000000001,
  'movie_relevance': 0.6827000304811133},
 {'movie_id': 4610,
  'movie_title': 'January Man, The',
  'movie_genres': 'Comedy, Crime, Mystery, Thriller',
  'movie_similarity': 0.5000000000000001,
  'movie_relevance': 0.6827000304811133},
 {'movie_id': 4898,
  'movie_title': 'Novocaine',
  'movie_genres': 'Comedy, Crime, Mystery, Thriller',
  'movie_similarity': 0.5000000000000001,
  'movie_relevance': 0.6827000304811133},
 {'movie_id': 5021,
  'movie_title': 'Murder by Death',
  'movie_genres': 'Comedy, Crime, M

In [14]:
candidates_list = refinedMyAlgo.get_relevance_score(recs=recs, references=references)
print(len(candidates_list))

for item in candidates_list[0:20]:
    print('relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

100
relevance: 0.7433642685875197, title:Baby, The
relevance: 0.6944099556775261, title:The Raid: Redemption
relevance: 0.6827000304811133, title:Clue
relevance: 0.6827000304811133, title:January Man, The
relevance: 0.6827000304811133, title:Novocaine
relevance: 0.6827000304811133, title:Murder by Death
relevance: 0.6827000304811133, title:Deathtrap
relevance: 0.6808348486346194, title:Just Cause
relevance: 0.6808348486346194, title:Underneath
relevance: 0.6808348486346194, title:True Crime
relevance: 0.6808348486346194, title:Absolute Power
relevance: 0.6808348486346194, title:Saboteur
relevance: 0.6808348486346194, title:Mortal Thoughts
relevance: 0.6808348486346194, title:Eyes of Laura Mars
relevance: 0.6808348486346194, title:Jennifer 8
relevance: 0.6808348486346194, title:Obsession
relevance: 0.6808348486346194, title:Cat o' Nine Tails, The (Gatto a nove code, Il)
relevance: 0.6718412600194088, title:Suicide Kings
relevance: 0.6718412600194088, title:11:14
relevance: 0.66869654470

In [13]:
dist_i_R = refinedMyAlgo.calc_distance_item_in_list(item=candidates_list[24],this_list=candidates_list[0:5])
dist_i_R

0.6669543809016186

In [102]:
print(refinedMyAlgo.calc_diversity_score(actual_list=candidates_list[0:5], candidates_list=candidates_list[10:20]))

[{'div_score': 0.5152068879803822, 'idx': 0}, {'div_score': 0.7100494050081605, 'idx': 1}, {'div_score': 0.7100494050081605, 'idx': 2}, {'div_score': 0.7100494050081605, 'idx': 3}, {'div_score': 0.7100494050081605, 'idx': 4}, {'div_score': 0.7100494050081605, 'idx': 5}, {'div_score': 0.7100494050081605, 'idx': 6}, {'div_score': 0.7100494050081605, 'idx': 7}, {'div_score': 0.7100494050081605, 'idx': 8}, {'div_score': 0.7100494050081605, 'idx': 9}]


In [14]:
my_candidates = candidates_list.copy()
final_recs = refinedMyAlgo.diversify_recs_list(recs=my_candidates)
for item in final_recs:
    print('relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

relevance: 0.6722114831090638, title:Vovka in the Kingdom of Far Far Away
relevance: 0.6686136914706732, title:Just Cause
relevance: 0.6655443188693326, title:Don't Be a Menace to South Central While Drinking Your Juice in the Hood
relevance: 0.6376398341285096, title:Princess and the Warrior, The (Krieger und die Kaiserin, Der)
relevance: 0.6555025187005001, title:Ninja Scroll (Jûbei ninpûchô)
relevance: 0.6686136914706732, title:Underneath
relevance: 0.6655443188693326, title:Striptease
relevance: 0.6600746872551528, title:Witness
relevance: 0.6534993064714851, title:Care Bears Movie, The
relevance: 0.6655443188693326, title:Carpool
